In [2]:
# !pip install uv
# !uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

In [3]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

In [4]:
# Import necessary modules from llama_index
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

# Create an llm object to use for the QueryEngine and the ReActAgent
# The OpenAI class is used to interact with the OpenAI API
# The model parameter specifies which model to use, in this case, "gpt-4"
llm = OpenAI(model="gpt-4")

In [5]:
import phoenix as px
session = px.launch_app()

c:\Users\stlp\.conda\envs\reactor-feb-25\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
# Import the LlamaIndexInstrumentor class from the openinference.instrumentation.llama_index module
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

# Import the register function from the phoenix.otel module
from phoenix.otel import register

# Register a tracer provider using the register function
tracer_provider = register()

# Instrument the LlamaIndex with the registered tracer provider
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

OpenTelemetry Tracing Details
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [7]:
try:
    # Create a storage context from the default settings, specifying the directory to persist data
    storage_context = StorageContext.from_defaults(
        persist_dir="history.txt"
    )
    # Load the index from the storage context
    history = load_index_from_storage(storage_context)

    # Set the flag to indicate that the index was successfully loaded
    index_loaded = True
except:
    # If an error occurs, set the flag to indicate that the index was not loaded
    index_loaded = False

In [8]:
if not index_loaded:
    # If the index is not loaded, load data from the specified file
    history_docs = SimpleDirectoryReader(
        input_files=["history.txt"]
    ).load_data()

    # Build the index from the loaded documents, showing progress during the process
    history_index = VectorStoreIndex.from_documents(history_docs, show_progress=True)
    
    # Persist the index to the specified directory
    history_index.storage_context.persist(persist_dir="./persisted_index/")

Generating embeddings: 100%|██████████| 5/5 [00:00<00:00,  7.96it/s]


In [9]:
# Convert the history_index to a query engine
# The query engine will use the top 3 most similar documents for each query
# The llm (language model) is used to process the queries and generate responses
history_engine = history_index.as_query_engine(similarity_top_k=3, llm=llm)

In [10]:
# Create a list of query engine tools
query_engine_tools = [
    # Create a QueryEngineTool object for the history engine
    QueryEngineTool(
        query_engine=history_engine,  # Use the history_engine defined earlier
        metadata=ToolMetadata(
            name="history",  # Name of the tool
            description=(
                "Provides comprehensive access to a patient's complete medical history, including detailed records of previous diagnoses, treatment plans, surgical interventions, medication regimens, allergies, immunizations, and other critical clinical information. "
                "This tool is designed to assist healthcare providers, particularly in emergency settings, by delivering precise, plain text responses based on well-formulated queries. "
                "When using this tool, please input a detailed question that clearly specifies the aspects of the medical history you require—such as cardiac events, medication changes, surgical history, or chronic disease management—"
                " to ensure the most relevant and targeted information is retrieved."
            ),  # Description of the tool
        ),
    )
]

In [11]:
# Create a ReActAgent from the provided tools
# The ReActAgent will use the query_engine_tools defined earlier
# The llm (language model) is used to process the queries and generate responses
# verbose=True enables detailed logging of the agent's actions
# max_turns=10 sets the maximum number of turns the agent can take in a single interaction
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [36]:
initial_query = (
    "Please provide the patient's full medical history, including details of previous diagnoses, treatment plans, surgical interventions, medication regimens, allergies, immunizations, and other critical clinical information in the following format only:\n"
        '{\n'
        '  "Name": "Provide the patient\'s full name",\n'
        '  "Age": "Provide the patient\'s age",\n'
        '  "Gender": "Provide the patient\'s gender",\n'
        '  "Allergies": "List all known allergies",\n'
        '  "Past Treatments": "Provide details of past treatments with corresponding dates",\n'
        '  "Current Medication": "List all current medications with amount and frequency",\n'
        '  "Existing Health Condition": "Detail any existing health conditions"\n'
        '}\n'
        '"Suggested Follow-Up Questions": ["Provide a list of 3 suggested follow-up questions"]'
    )

In [47]:

from llama_index.core.agent import ReActAgent



# =============================================================================
# Function: handle_agent_response
# =============================================================================
def handle_agent_response(agent: ReActAgent, response: str) -> None:
    """
    Handles the agent's response by printing it and checking if the conversation is complete.
    If not complete, it prompts the user for the next input and continues the conversation.
    
    Args:
        agent (ReActAgent): The conversation agent.
        response (str): The current response from the agent.
    """
    print("Agent:", response)
    
    # if agent.is_conversation_complete():
    #     print("Conversation complete.")
    # else:
    #     pass
    #     # # Prompt the user for further input
    #     # user_input = input("User: ")
    #     # # Continue the conversation with the user's input
    #     # new_response = agent.continue_chat(user_input)
    #     # handle_agent_response(agent, new_response)

# =============================================================================
# Function: start_conversation
# =============================================================================
def start_conversation(agent: ReActAgent, initial_query: str) -> None:
    """
    Starts a conversation with the agent using the provided initial query.
    
    The initial query should be a detailed question in legitimate JSON format requesting the following details:
      - Name
      - Age
      - Gender
      - Allergies
      - Past Treatments with dates
      - Current Medication
      - Existing Health Condition
    
    Args:
        agent (ReActAgent): The conversation agent.
        initial_query (str): The initial query in plain text (formatted as valid JSON) to begin the conversation.
    """
    response = agent.chat(initial_query)
    return response
    # handle_agent_response(agent, response)

# =============================================================================
# Function: handle_user_query
# =============================================================================

def handle_user_query(agent: ReActAgent, user_query: str, suggestions: bool = True) -> None:

# Define the prompt for the agent including the request for follow-up questions in a static JSON format
    prompt = f"""
    {user_query}
    Please respond in the following JSON format:
    """ + """{
        "response_to_user_query": "<agent response to the user query>",
        "suggested_follow_up": ["question1", "question2", "question3"]
    }""" if {suggestions} else """{
        "response_to_user_query": "<agent response to the user query>"
    }"""
    return prompt
    # Get the response from the agent
    response = agent.chat(prompt)

    return response
    # Handle the agent's response
    # handle_agent_response(agent, response)


In [38]:
response = start_conversation(agent, initial_query)

> Running step 26f43d9b-6732-4a30-9105-f550c57fc9d5. Step input: Please provide the patient's full medical history, including details of previous diagnoses, treatment plans, surgical interventions, medication regimens, allergies, immunizations, and other critical clinical information in the following format only:
{
  "Name": "Provide the patient's full name",
  "Age": "Provide the patient's age",
  "Gender": "Provide the patient's gender",
  "Allergies": "List all known allergies",
  "Past Treatments": "Provide details of past treatments with corresponding dates",
  "Current Medication": "List all current medications with amount and frequency",
  "Existing Health Condition": "Detail any existing health conditions"
}
"Suggested Follow-Up Questions": ["Provide a list of 3 suggested follow-up questions"]
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: history
Action Input: {'input': "Provide the patient's full medical his

In [39]:
import json
response = json.loads(response.response)

print(response)

{'Name': 'Johnathan A. Doe', 'Age': '58', 'Gender': 'Male', 'Allergies': 'No known drug allergies. Seasonal environmental allergies to pollen.', 'Past Treatments': {'Appendectomy': '1978', 'Coronary artery bypass grafting': '2018', 'Cataract extraction with intraocular lens implantation in right eye': '2016', 'Inguinal hernia repair': '1995', 'Minor dermatologic excision': '2010'}, 'Current Medication': {'Lisinopril': '20 mg once daily', 'Metformin': '1000 mg twice daily', 'Simvastatin': '40 mg once daily', 'Aspirin': '81 mg once daily', 'Metoprolol Succinate': '50 mg once daily', 'Omeprazole': '20 mg once daily', 'Tamsulosin': '0.4 mg once daily', 'Ibuprofen': '200 mg as needed', 'Daily multivitamin': '1 tablet daily'}, 'Existing Health Condition': {'Hypertension': 'Diagnosed in 2005', 'Type 2 diabetes mellitus': 'Diagnosed in 2010', 'Hyperlipidemia': 'Diagnosed in 2008', 'Coronary artery disease': 'Diagnosed prior to 2018', 'Gastroesophageal reflux disease': 'Diagnosed in 2012', 'Obs

In [44]:
json.loads(handle_user_query(agent, response["Suggested Follow-Up Questions"][1]).response)

> Running step 62bfe2bb-c38c-4eff-956a-0d11031152e4. Step input: 
    What was the outcome of the patient's recent hospital admission?
    Please respond in the following JSON format:
    {
        "response_to_user_query": "<agent response to the user query>",
        "suggested_follow_up": ["question1", "question2", "question3"]
    }
    
Thought: (Implicit) I can answer without any more tools!
Answer: {
  "response_to_user_query": "The patient was admitted to the hospital for evaluation of worsening exertional chest pain and dyspnea, with the aim to rule out acute ischemic events and optimize management of multiple chronic conditions. During the hospital stay, the patient's cardiac condition was monitored, and adjustments were made to their beta-blocker therapy. A comprehensive stress test revealed moderate ischemia, which prompted further cardiologic evaluation. The patient's blood pressure and glucose levels were also closely monitored, with minor medication adjustments made duri

{'response_to_user_query': "The patient was admitted to the hospital for evaluation of worsening exertional chest pain and dyspnea, with the aim to rule out acute ischemic events and optimize management of multiple chronic conditions. During the hospital stay, the patient's cardiac condition was monitored, and adjustments were made to their beta-blocker therapy. A comprehensive stress test revealed moderate ischemia, which prompted further cardiologic evaluation. The patient's blood pressure and glucose levels were also closely monitored, with minor medication adjustments made during the stay. Pain management for osteoarthritis was optimized with scheduled NSAIDs and a physical therapy consultation. The patient's renal function and electrolyte balance were monitored due to their stage 2 Chronic Kidney Disease (CKD). The patient was discharged with a plan for home-based cardiac rehabilitation, strict dietary modifications, an exercise regimen, and education on recognizing signs of acute

In [50]:
handle_user_query(agent, response["Suggested Follow-Up Questions"][1], suggestions=False)

'\n    What was the outcome of the patient\'s recent hospital admission?\n    Please respond in the following JSON format:\n    {\n        "response_to_user_query": "<agent response to the user query>",\n        "suggested_follow_up": ["question1", "question2", "question3"]\n    }'